In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
import time

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HUGGINGFACE_API_KEY")
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')

Web-scraper

In [2]:
import requests

def scrape_jina_ai(url: str):
  response = requests.get("https://r.jina.ai/" + url)
  return response.text

print(requests.get("https://s.jina.ai/https://www.w3schools.com/python/ref_string_format.asp").text)

[1] Title: 7. Input and Output
[1] URL Source: https://docs.python.org/3/tutorial/inputoutput.html
[1] Description: To use <strong>formatted</strong> <strong>string</strong> literals, begin a <strong>string</strong> with f or F before the opening quotation mark or triple quotation mark. Inside this <strong>string</strong>, you can write a <strong>Python</strong> expression between { and } characters that can refer to variables or literal values.
[1] Markdown Content:
There are several ways to present the output of a program; data can be printed in a human-readable form, or written to a file for future use. This chapter will discuss some of the possibilities.

7.1. Fancier Output Formatting[¶](https://docs.python.org/3/tutorial/inputoutput.html#fancier-output-formatting "Link to this heading")
---------------------------------------------------------------------------------------------------------------------------------------

So far we’ve encountered two ways of writing values: _expre

Declare the web search

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.retrievers import TavilySearchAPIRetriever
web_search_tool = TavilySearchResults(k=1)

def search_function(question : str):
    # web_search_tool = TavilySearchAPIRetriever(k=5, search_depth="advanced", max_results=5)
    x = web_search_tool.invoke(question)
    url_response = ""
    for i in x:
        url_response += scrape_jina_ai(i['url'])
    return url_response

In [18]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided. If the answer is not in the context,
    respond with 'I do not know'.
    You are forbidden from answering any question unrelated to Stock Exchange and the Stock Market.
    The context you are given is data scraped from the web about the user question related to stocks.

Context: {context}

Question: {question}

If date and time is required, consider the date and time as {dates} in %Y-%m-%d %H:%M:%S format."""
)

question = "What is Nvidia stock price today ?"
url_response = search_function(question)

In [25]:
import datetime

now = datetime.datetime.now()
dates = now.strftime("%Y-%m-%d %H:%M:%S")
dates

'2024-06-26 10:53:26'

In [7]:
# ### OPTIONAL TRIAL
# result = requests.get("https://s.jina.ai/" + question).text

In [20]:
url_response

'Title: NVIDIA Corporation (NVDA) Stock Price, News, Quote & History - Yahoo Finance\n\nURL Source: https://finance.yahoo.com/quote/NVDA/\n\nMarkdown Content:\n*   [Today\'s news](https://www.yahoo.com/news/)\n*   [US](https://www.yahoo.com/news/us/)\n*   [Politics](https://www.yahoo.com/news/politics/)\n*   [World](https://www.yahoo.com/news/world/)\n*   [Tech](https://www.yahoo.com/tech/)\n    \n    *   [Reviews and deals](https://www.yahoo.com/tech/reviews-deals/)\n    *   [Audio](https://www.yahoo.com/tech/audio/)\n    *   [Computing](https://www.yahoo.com/tech/computing/)\n    *   [Gaming](https://www.yahoo.com/tech/gaming/)\n    *   [Health](https://www.yahoo.com/tech/health/)\n    *   [Home](https://www.yahoo.com/tech/home/)\n    *   [Phones](https://www.yahoo.com/tech/phones/)\n    *   [Science](https://www.yahoo.com/tech/science/)\n    *   [TVs](https://www.yahoo.com/tech/tvs/)\n*   [Climate change](https://www.yahoo.com/tagged/climate-change/)\n*   [Health](https://www.yahoo.

In [21]:
## Data infestion

from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
## Data infestion

from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain.schema.document import Document
from langchain_community.vectorstores import Chroma
def get_text_chunks_langchain(url_response):
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200,separators="\n")
    docs = [Document(page_content=x) for x in text_splitter.split_text(url_response)]
    return docs

docs = get_text_chunks_langchain(url_response)
# docs = get_text_chunks_langchain(result)
documents = filter_complex_metadata(docs)
documents

[Document(page_content="Title: NVIDIA Corporation (NVDA) Stock Price, News, Quote & History - Yahoo Finance\n\nURL Source: https://finance.yahoo.com/quote/NVDA/\n\nMarkdown Content:\n*   [Today's news](https://www.yahoo.com/news/)\n*   [US](https://www.yahoo.com/news/us/)\n*   [Politics](https://www.yahoo.com/news/politics/)\n*   [World](https://www.yahoo.com/news/world/)\n*   [Tech](https://www.yahoo.com/tech/)\n    \n    *   [Reviews and deals](https://www.yahoo.com/tech/reviews-deals/)"),
 Document(page_content='*   [World](https://www.yahoo.com/news/world/)\n*   [Tech](https://www.yahoo.com/tech/)\n    \n    *   [Reviews and deals](https://www.yahoo.com/tech/reviews-deals/)\n    *   [Audio](https://www.yahoo.com/tech/audio/)\n    *   [Computing](https://www.yahoo.com/tech/computing/)\n    *   [Gaming](https://www.yahoo.com/tech/gaming/)\n    *   [Health](https://www.yahoo.com/tech/health/)\n    *   [Home](https://www.yahoo.com/tech/home/)\n    *   [Phones](https://www.yahoo.com/tec

In [22]:
## Vector embeddings and Vector Store
from langchain_community.embeddings import HuggingFaceEmbeddings
ollama_emb = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma()
try:
    db._collection.delete(db.get()['ids'])
    db = Chroma.from_documents(documents, ollama_emb)
except:
    db = Chroma.from_documents(documents, ollama_emb)

e:\Talrop\LLMs\venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [23]:
result = db.similarity_search(question)
result

[Document(page_content="On Friday 06/21/2024 the closing price of the NVIDIA Corp. share was $118.06 on BTT. Compared to the opening price on Friday 06/21/2024 on BTT of $129.20, this is a drop of 9.44%. NVIDIA Corp.'s market capitalization is $3,115.94 B by 24.60 B shares outstanding."),
 Document(page_content='Nvidia Stock Price History by Markets Insider\n---------------------------------------------'),
 Document(page_content='NVIDIA Stock Snapshot\n---------------------\n\n118.06\n\nBid\n\n250.00\n\nBid Size\n\n118.10\n\nAsk\n\n100.00\n\nAsk Size\n\n6/24/2024\n\nDate\n\n7:59 PM\n\nTime\n\n0.00\n\nVolume\n\n118.06\n\nPrev. Close\n\n0.00\n\nOpen\n\n3,117.69 B\n\nMarket Cap in USD\n\n24.60 B\n\nNumber of Shares\n\n39.23\n\n52 Week Low\n\n140.82\n\n52 Week High\n\n116.58\n\n0.02\n\nDividend in USD\n\n0.03\n\nDividend Yield\n\n51.14\n\nP/E Ratio\n\n96.01\n\nFree Float in %\n\n1.21\n\nEPS in USD\n\n1.74\n\nBook Value per Share in USD\n\n1.13\n\nCash Flow per Share in USD'),
 Document(pag

In [24]:
chain = (
    prompt
    | ChatGroq(model="llama3-70b-8192", temperature=0)
    | StrOutputParser()
)

chain.invoke({"question": question, "context": result, "dates": dates})

'According to the context, the latest available stock price of NVIDIA Corp. is $118.06, which was the closing price on Friday 06/21/2024.'